In [3]:
!pip install pyrsgis


In [4]:
import os, math, numpy as np
from pyrsgis import raster, convert
from osgeo import gdal
from google.colab import drive
from copy import deepcopy
import random
import matplotlib.pyplot as plt

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
f2017 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/Sentinel_Bangalore_2017.tif"
f2018 = "/content/drive/MyDrive/Capstone_Data/Sentinel/2018.tif"
f2019 = "/content/drive/MyDrive/Capstone_Data/Sentinel/2019.tif"
f2020 = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/Sentinel_Bangalore_2020.tif"
f2021 = "/content/drive/MyDrive/Capstone_Data/Sentinel/2021.tif"
f2022= "/content/drive/MyDrive/Capstone_Data/Bangalore_data/Sentinel_Bangalore_2023.tif"
f2023= '/content/drive/MyDrive/Capstone_Data/Sentinel/predicted_2023.tif'
road_network_path = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/road.tif"
pop = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/pop.tif"
metro = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/metro.tif"
school = "/content/drive/MyDrive/Capstone_Data/Bangalore_data/metro.tif"

ds1, arr1 = raster.read(f2017,bands=1)
ds2, arr2 = raster.read(f2018,bands=1)
ds3, arr3 = raster.read(f2019,bands=1)
ds4,arr4 = raster.read(f2020,bands =1)
ds5, arr5 = raster.read(f2021,bands=1)
ds6, arr6 = raster.read(f2022,bands=1)
ds7, arr7 = raster.read(f2023,bands=1)
road_ds,road_arr = raster.read(road_network_path,bands=1)
pop_ds, pop_arr = raster.read(pop,bands=1)
metro_ds,metro_arr = raster.read(metro,bands=1)

In [7]:
x=list()
y=list()
for i in range(arr1.shape[0]):
  for j in range(arr1.shape[1]):
    val=None
    if arr1[i,j]==1:
      val=[1,0,0,0]
    elif arr1[i,j]==2:
      val=[0,1,0,0]
    elif arr1[i,j]==3:
      val=[0,0,1,0]
    else:
      val=[0,0,0,1]
    val.append(road_arr[i,j])
    val.append(pop_arr[i,j])
    val.append(metro_arr[i,j])
    x.append(val)
    val=None
    if arr4[i,j]==1:
      val=[1,0,0,0]
    elif arr4[i,j]==2:
      val=[0,1,0,0]
    elif arr4[i,j]==3:
      val=[0,0,1,0]
    else:
      val=[0,0,0,1]
    y.append(val)
x=np.array(x)
y=np.array(y)

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sc
import random
import time
from sklearn.utils import shuffle
from keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

In [32]:
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size = 0.001, random_state = 0)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((1488591, 7), (1488591, 4), (1491, 7), (1491, 4))

In [33]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [34]:
model = Sequential()
model.add(Dense(8, input_dim = 7, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(test_x,test_y, epochs = 5, batch_size = 5)
scores = model.evaluate(train_x, train_y)


Epoch 1/5
299/299 [==============================] - 2s 3ms/step - loss: 76.4716
Epoch 2/5
299/299 [==============================] - 1s 3ms/step - loss: 7.2149
Epoch 3/5
299/299 [==============================] - 1s 3ms/step - loss: 6.7432
Epoch 4/5
299/299 [==============================] - 1s 2ms/step - loss: 6.1486
Epoch 5/5
46519/46519 [==============================] - 88s 2ms/step - loss: 3.3430


In [36]:
x=model.predict(test_x)

47/47 [==============================] - 0s 2ms/step


In [41]:
ans=list()
for i in x:
  i=list(i)
  max_value = max(i)
  max_index = i.index(max_value)
  ans.append(max_index+1)
print(ans)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 